In [1]:
!pip install -q pyspellchecker
!pip install -q ktrain

In [50]:
import pandas as pd 
import numpy as np 
import re
import string 
import spacy
import xgboost as xgb
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer, ViTFeatureExtractor, ViTModel
import tensorflow as tf
import tensorflow.keras.backend as K
import lightgbm as lgb
import warnings
from PIL import Image
from spellchecker import SpellChecker
from tensorflow.keras import activations

warnings.filterwarnings("ignore")

nlp = spacy.load("en_core_web_sm")

In [51]:
Task1_Train = pd.read_csv("/kaggle/input/subtask-c-final/SubTask-C-train.csv")
Task1_Test = pd.read_csv("/kaggle/input/subtask-c/SubTask-C(indextweet)test.csv")

In [52]:
Task1_Train['label'].value_counts()

label
1    4328
3    2256
2     700
Name: count, dtype: int64

In [53]:
Task1_Train.dropna(inplace = True)

In [54]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # Emoticons
                               u"\U0001F300-\U0001F5FF"  # Symbols & Pictographs
                               u"\U0001F680-\U0001F6FF"  # Transport & Map Symbols
                               u"\U0001F700-\U0001F77F"  # Alphanumeric Supplement
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251" 
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_punctuation(input_string):
    translator = str.maketrans('', '', string.punctuation)
    
    result = input_string.translate(translator)
    
    return result

def remove_non_english_chars(input_string):
    translation_table = dict.fromkeys(
        i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('C')
    )

    result = input_string.translate(translation_table)

    return result

def correct_spelling(input_text):
    try:
        # Create a SpellChecker object
        spell = SpellChecker()

        # Split the input text into words
        words = input_text.split()

        # Find misspelled words
        misspelled = spell.unknown(words)

        # Correct misspelled words
        corrected_text = " ".join(spell.correction(word) if word in misspelled else word for word in words)

        return corrected_text

    except Exception as e:
        print(f"An error occurred: {e}")
        return input_text
    
hyperlink_pattern = r'https?://\S+|www\.\S+'

def PreProcessTweets(tweet):
  tweet2 = str(tweet)
  tweet2 = re.sub(r'^RT[\s]+', '', tweet)
  tweet2 = re.sub(r'#\w+', '', tweet2)
  tweet2 = tweet2.replace("LINK","")
  tweet2 = tweet2.replace("&amp","") 
  tweet2 = re.sub(r'@\w+', '', tweet2)
  tweet2 = tweet2.replace("\n","")
  tweet2 = re.sub(r'_', ' ', tweet2)
  tweet2 = tweet2.replace("UN","United Nations ")
  tweet2 = tweet2.replace("imglipcom","")
  tweet2 = re.sub(hyperlink_pattern, '', tweet2)
  tweet2 = tweet2.replace(":","") 
  tweet2 = remove_emojis(tweet2)
  tweet2 = remove_punctuation(tweet2)
  tweet2 = re.sub(' +', ' ',tweet2)
  tweet2 = tweet2.replace("hes","he is")
  tweet2 = tweet2.replace("govt","Government")
  #tweet2 = correct_spelling(tweet2)
  return tweet2

Task1_Train['tweet'] = Task1_Train['tweet'].apply(lambda x: PreProcessTweets(x))
Task1_Test['tweet'] = Task1_Test['tweet'].apply(lambda x: PreProcessTweets(x))

In [26]:
def NERFeatureExtractor(text, nlp = nlp):
    doc = nlp(text)
    PersonCount = NORPCount = ORGCount = 0
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            PersonCount += 1
        elif ent.label_ == 'NORP':
            NORPCount += 1
        elif ent.label_ == 'ORG':
            ORGCount += 1
    NERFeaturesReady = {'PersonCount' : PersonCount, 'NORPCount' : NORPCount , 'ORGCount' : ORGCount}
    return NERFeaturesReady

In [55]:
Task1_Train['label'] = Task1_Train['label'].astype(str)

def ReSampler(reqClass, DataFrame, TargetClass = 'label'):
    SubDataFrame = DataFrame[DataFrame[TargetClass] == reqClass].copy()
    DataFrame = pd.concat([DataFrame, SubDataFrame], axis = 0)
    del SubDataFrame
    return DataFrame

Task1_Train = ReSampler('2',Task1_Train)
Task1_Train['label'].value_counts()

label
1    4328
3    2256
2    1400
Name: count, dtype: int64

In [9]:
NERFeatureExtractor('british Government msm sheer hypocrisy Out of respect you must stop strikingprotestingplaying sport GovtSystemsChangePlanetarySurvivalCitizensAssembliesOurOnlyHope')

{'PersonCount': 0, 'NORPCount': 0, 'ORGCount': 1}

In [ ]:
# Use a pipeline as a high-level helper
Task1_Test['label'] = -1

from transformers import pipeline

pipe = pipeline("summarization", model="currentlyexhausted/flan-t5-summarizer") 

Task1_Test['result'] = Task1_Test['text'].apply(pipe)

In [ ]:
for i in range(len(Task1_Train)):
    NERFeaturesReady = NERFeatureExtractor(Task1_Train['text'][i])
    Task1_Train['PersonCount'][i] = NERFeaturesReady['PersonCount']
    Task1_Train['NORPCount'][i] = NERFeaturesReady['NORPCount']
    Task1_Train['ORGCount'][i] = NERFeaturesReady['ORGCount']

for i in range(len(Task1_Test)):
    NERFeaturesReady = NERFeatureExtractor(Task1_Dev['text'][i])
    Task1_Test['PersonCount'][i] = NERFeaturesReady['PersonCount']
    Task1_Test['NORPCount'][i] = NERFeaturesReady['NORPCount']
    Task1_Test['ORGCount'][i] = NERFeaturesReady['ORGCount']

In [57]:
def dice_loss(smooth=1e-5, from_logits=True):
    def dice_loss(y_true, y_pred,):
        y_true = K.cast(y_true, dtype=tf.float32)
        y_pred = K.cast(y_pred, dtype=tf.float32)
        intersection = tf.reduce_sum(y_true * y_pred)
        sum_of_squares_pred = tf.reduce_sum(tf.square(y_pred))
        sum_of_squares_true = tf.reduce_sum(tf.square(y_true))
        dice = 1. - (2. * intersection + smooth) / (sum_of_squares_pred + sum_of_squares_true + smooth)
        return dice
    return dice_loss

In [75]:
def focal_loss(gamma=3, alpha=0.25, from_logits=True):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        epsilon = 1.e-9
        y_true = tf.cast(y_true, dtype=tf.float32)
        y_pred = tf.cast(y_pred, dtype=tf.float32)
        if from_logits:
            y_pred = activations.softmax(y_pred)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.math.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed

In [14]:
from sklearn.utils import class_weight

# Calculate the weights for each class so that we can balance the data
weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                            classes = np.unique(Task1_Train['label']),
                                            y = Task1_Train['label'])
weights


array([0.56099815, 3.46857143, 1.07624113])

In [56]:
X_train, X_test, y_train, y_test = train_test_split(Task1_Train['tweet'], Task1_Train['label'], test_size=0.15, random_state = 44)

In [62]:
import ktrain
from ktrain import text

model_name = "roberta-large"

t = text.Transformer(model_name, maxlen = 40, classes = ['1','2','3'])
trn = t.preprocess_train(X_train.values, y_train.values)
val = t.preprocess_test(X_test.values, y_test.values)

model = t.get_classifier()

model.compile(loss = dice_loss(smooth = 1e-6), metrics = ['accuracy'], optimizer = 'adam')

learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size = 16)

preprocessing train...
language: en
train sequence lengths:
	mean : 18
	95percentile : 38
	99percentile : 45


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 18
	95percentile : 38
	99percentile : 43


In [63]:
learner.autofit(2e-5, monitor='val_loss')

early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 2e-05...
Epoch 1/1024
425/425 [==============================] - 176s 331ms/step - loss: 0.3034 - accuracy: 0.6490 - val_loss: 0.2558 - val_accuracy: 0.6694
Epoch 2/1024
425/425 [==============================] - 130s 307ms/step - loss: 0.2673 - accuracy: 0.6637 - val_loss: 0.2219 - val_accuracy: 0.7028
Epoch 3/1024
425/425 [==============================] - 130s 306ms/step - loss: 0.2465 - accuracy: 0.6846 - val_loss: 0.2138 - val_accuracy: 0.7270
Epoch 4/1024
425/425 [==============================] - 130s 306ms/step - loss: 0.2322 - accuracy: 0.7081 - val_loss: 0.2078 - val_accuracy: 0.7295
Epoch 5/1024
425/425 [==============================] - 130s 306ms/step - loss: 0.2211 - accuracy: 0.7252 - val_loss: 0.2048 - val_accuracy: 0.7295
Epoch 6/1024
425/425 [==============================] - 129s 304ms/step - lo

In [64]:
import json 

predictor = ktrain.get_predictor(learner.model, preproc=t)

with open('submission.json', 'w') as json_file:
    for i in range(len(Task1_Test)):
        pred = predictor.predict(str(Task1_Test['tweet'][i]))
        predictionsDictionary = {'index': int(Task1_Test['index'][i]), 'prediction': int(pred)}
        json.dump(predictionsDictionary, json_file)    
        json_file.write('\n')      

In [57]:
import pandas as pd 
Results1 = pd.read_json("/kaggle/input/qwqqwwqwq/submission/submission.json", lines=True)
Results2 = pd.read_json("/kaggle/input/qwqqwwqwq/submission(1)/submission.json", lines=True)
Results3 = pd.read_json("/kaggle/input/qwqqwwqwq/submission(2)/submission.json", lines=True)

In [ ]:
FinalPreds=[]
for i in range(len(Results1)):
  preds = {'0': 0, '1': 0}
  preds[str(Results1['prediction'][i])] +=1
  preds[str(Results2['prediction'][i])] +=1
  preds[str(Results3['prediction'][i])] +=1
  # Convert the dictionary to a list of tuples.
  list_of_tuples = list(preds.items())

  # Sort the list of tuples by the value of the tuple.
  sorted_list_of_tuples = sorted(list_of_tuples, key=lambda x: x[1])
  # Return the first element of the sorted list.
  key_of_largest_item = sorted_list_of_tuples[len(preds) - 1][0]
  FinalPreds.append(int(key_of_largest_item))